In [1]:
%pip install -U langchain-community langgraph langchain-anthropic tavily-python langgraph-checkpoint-sqlite

   ---------------------------------------- 0.0/894.7 kB ? eta -:--:--
   ---------------------------------------- 894.7/894.7 kB 7.4 MB/s eta 0:00:00

   ------------------------- -------------- 5/8 [anthropic]
   ------------------------- -------------- 5/8 [anthropic]
   ------------------------- -------------- 5/8 [anthropic]
   ------------------------- -------------- 5/8 [anthropic]
   ----------------------------------- ---- 7/8 [langgraph-checkpoint-sqlite]
   ---------------------------------------- 8/8 [langgraph-checkpoint-sqlite]

Note: you may need to restart the kernel to use updated packages.


In [15]:
import getpass
import os

try:
    # load environment variables from .env file (requires `python-dotenv`)
    from dotenv import load_dotenv

    load_dotenv()
except ImportError:
    pass

os.environ["LANGSMITH_TRACING"] = "true"
if "LANGSMITH_API_KEY" not in os.environ:
    os.environ["LANGSMITH_API_KEY"] = getpass.getpass(
        prompt="Enter your LangSmith API key (optional): "
    )
if "LANGSMITH_PROJECT" not in os.environ:
    os.environ["LANGSMITH_PROJECT"] = getpass.getpass(
        prompt='Enter your LangSmith Project Name (default = "default"): '
    )
    if not os.environ.get("LANGSMITH_PROJECT"):
        os.environ["LANGSMITH_PROJECT"] = "default"

In [16]:
import getpass
import os

os.environ["TAVILY_API_KEY"] = getpass.getpass()

 ········


In [5]:
from langchain_community.tools.tavily_search import TavilySearchResults

search = TavilySearchResults(max_results=2)
search_results = search.invoke("what is the weather in SF")
print(search_results)
# If we want, we can create other tools.
# Once we have all the tools we want, we can put them in a list that we will reference later.
tools = [search]

[{'title': 'Thursday, May 22, 2025. San Francisco, CA - Weather Forecast', 'url': 'https://weathershogun.com/weather/usa/ca/san-francisco/480/may/2025-05-22', 'content': 'San Francisco, California Weather: Thursday, May 22, 2025. Cloudy weather, overcast skies with clouds. Day 70°. Night 57°.', 'score': 0.9437689}, {'title': 'Weather in San Francisco in May 2025 - Detailed Forecast', 'url': 'https://www.easeweather.com/north-america/united-states/california/city-and-county-of-san-francisco/san-francisco/may', 'content': 'Sunny\n| 66° /50° | 0\xa0in | 5 |  |\n| May 22 | \nSunny\n| 66° /48° | 0\xa0in | 6 |  |\n| May 23 | \nSunny\n| 64° /50° | 0\xa0in | 5 |  |\n| May 24 | \nSunny\n| 68° /50° | 0\xa0in | 5 |  |\n| May 25 | \nPartly cloudy\n| 66° /50° | 0\xa0in | 5 |  |\n| May 26 | \nSunny\n| 62° /50° | 0\xa0in | 5 |  |\n| May 27 | \nSunny\n| 64° /50° | 0\xa0in | 5 |  |\n| May 28 | \nPatchy rain possible\n| 62° /50° | 0\xa0in | 4 |  |\n| May 29 | \nCloudy\n| 64° /48° | 0\xa0in | 5 |  |\n| M

In [21]:
from langchain_ollama import ChatOllama

model = ChatOllama(
    model="qwen3:4b"
)

In [22]:
from langchain_core.messages import HumanMessage

response = model.invoke([HumanMessage(content="hi!")])
response.content

'<think>\nOkay, the user said "hi!" which is a greeting. I need to respond politely. Let me start by acknowledging their greeting. Maybe say "Hello!" to keep it friendly.\n\nNow, I should check if they need anything specific. Since they just said hi, maybe they\'re looking for help or just want to chat. I should offer assistance. Let me ask how I can help them today.\n\nI should keep the tone open and welcoming. Avoid any technical jargon. Make sure the response is concise but inviting. Let me put that together: "Hello! How can I assist you today?" That sounds good. Let me make sure there are no typos and that it\'s natural. Yep, that works.\n</think>\n\nHello! How can I assist you today? 😊'

In [23]:
model_with_tools = model.bind_tools(tools)

In [24]:
response = model_with_tools.invoke([HumanMessage(content="Hi!")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: <think>
Okay, the user just said "Hi!" I need to respond appropriately. Let me check the tools available. There's a search function, but the user didn't ask a question that requires it. Since they just greeted me, I should respond with a friendly greeting and ask how I can assist them. No need to call any functions here. Just a standard reply.
</think>

Hello! How can I assist you today? 😊
ToolCalls: []


In [25]:
response = model_with_tools.invoke([HumanMessage(content="What's the weather in SF?")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: 
ToolCalls: [{'name': 'tavily_search_results_json', 'args': {'query': 'weather in San Francisco'}, 'id': 'a2fc21f4-9a7e-4beb-a6e7-a28bd395c972', 'type': 'tool_call'}]


In [26]:
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(model, tools)

In [27]:
response = agent_executor.invoke({"messages": [HumanMessage(content="hi!")]})

response["messages"]

[HumanMessage(content='hi!', additional_kwargs={}, response_metadata={}, id='1ddb107d-b8a0-45f8-96d8-4353c1f22848'),
 AIMessage(content='<think>\nOkay, the user sent "hi!" which is a greeting. I need to respond appropriately. Since there\'s no specific question or request here, I shouldn\'t call any functions. My job is to acknowledge their greeting politely. Let me make sure there\'s nothing implied in their message that requires a tool. They just said hello, so a simple response like "Hello! How can I assist you today?" should work. No need for any tool calls here.\n</think>\n\nHello! How can I assist you today?', additional_kwargs={}, response_metadata={'model': 'qwen3:4b', 'created_at': '2025-05-22T18:03:40.605574963Z', 'done': True, 'done_reason': 'stop', 'total_duration': 4968904477, 'load_duration': 23849103, 'prompt_eval_count': 173, 'prompt_eval_duration': 341164722, 'eval_count': 106, 'eval_duration': 4603120198, 'model_name': 'qwen3:4b'}, id='run--0f599157-f272-476f-9dbe-b59

In [28]:
response = agent_executor.invoke(
    {"messages": [HumanMessage(content="whats the weather in sf?")]}
)
response["messages"]

[HumanMessage(content='whats the weather in sf?', additional_kwargs={}, response_metadata={}, id='9d39b342-b2f1-4e32-bcca-af2a2f9f9bee'),
 AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'qwen3:4b', 'created_at': '2025-05-22T18:05:26.723631983Z', 'done': True, 'done_reason': 'stop', 'total_duration': 10871784811, 'load_duration': 28230823, 'prompt_eval_count': 178, 'prompt_eval_duration': 697584770, 'eval_count': 237, 'eval_duration': 10144009812, 'model_name': 'qwen3:4b'}, id='run--932f3662-be26-45ae-8b85-268f36c0afc5-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'weather in sf'}, 'id': '11e62ba4-6a3a-4337-a53f-a582dd8fa1af', 'type': 'tool_call'}], usage_metadata={'input_tokens': 178, 'output_tokens': 237, 'total_tokens': 415}),
 ToolMessage(content='[{"title": "Thursday, May 22, 2025. San Francisco, CA - Weather Forecast", "url": "https://weathershogun.com/weather/usa/ca/san-francisco/480/may/2025-05-22", "content": "San Francisco, C

In [29]:
for step in agent_executor.stream(
    {"messages": [HumanMessage(content="whats the weather in sf?")]},
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

whats the weather in sf?
================================== Ai Message ==================================
Tool Calls:
  tavily_search_results_json (a8ab0a5a-0080-4faa-b291-7b725c4a9029)
 Call ID: a8ab0a5a-0080-4faa-b291-7b725c4a9029
  Args:
    query: whats the weather in sf?
================================= Tool Message =================================
Name: tavily_search_results_json

[{"title": "Thursday, May 22, 2025. San Francisco, CA - Weather Forecast", "url": "https://weathershogun.com/weather/usa/ca/san-francisco/480/may/2025-05-22", "content": "San Francisco, California Weather: Thursday, May 22, 2025. Cloudy weather, overcast skies with clouds. Day 70°. Night 57°.", "score": 0.96724063}, {"title": "Weather in San Francisco in May 2025 - Detailed Forecast", "url": "https://www.easeweather.com/north-america/united-states/california/city-and-county-of-san-francisco/san-francisco/may", "content":

In [30]:
for step, metadata in agent_executor.stream(
    {"messages": [HumanMessage(content="whats the weather in sf?")]},
    stream_mode="messages",
):
    if metadata["langgraph_node"] == "agent" and (text := step.text()):
        print(text, end="|")

<think>
Okay, the user asked for the weather in San Francisco. I need to check the tool response provided. The first result mentions Thursday, May 22, 2025, with cloudy and overcast skies, day temperature 70°, night 57°. The second result is from world-weather.info, showing May 22 as a day with +52°, but it's a bit unclear. Wait, the second result's content says "Thursday, 22 May. +52°." but the first result says day 70°. Hmm, maybe there's a discrepancy. The first source is from weathershogun.com, which might be more specific. The score for the first result is higher (0.95 vs 0.84), so it's more reliable. The user might want the most accurate info. So I should present the first result's details, mentioning cloudy with day temperature 70° and night 57°. Also, note that the second source mentions +52°, but since the first is more detailed, I'll go with that. Make sure to state the date and the conditions clearly.
</think>

The weather in San Francisco on Thursday, May 22, 2025, is expec

In [31]:
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()

In [32]:
agent_executor = create_react_agent(model, tools, checkpointer=memory)

config = {"configurable": {"thread_id": "abc123"}}